In [ ]:
import torch
import torch.utils.data as Data
from rule.GobangBase import * 
import RL.RLNet as RL
import RL.DataCreate as DC
import RL.DataLoader as DL
import time
import torch.nn as nn
from torchsummary import summary

In [ ]:
# net = RL.RLNet()
#print(net)
train_times = 500
lr = 0.01
epochs_num = 10
dataCreator = DC.DataCreator()
optim = torch.optim.Adam(net.parameters(),lr=lr)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def train(net,data_iterator,iswin,optim,device,num_epochs):
    net = net.to(device)
    print("train on ",device)
    loss = torch.nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        
        for X,y in data_iterator:
            X = X.to(device)
            y = y.to(device)
            label = torch.zeros((len(y)),device=device,dtype=torch.long)
            for i in range(len(y)):
                label[i] = int(y[i][0]*15 + y[i][1])
            y_hat = net(X)
            l = loss(y_hat,label)
            optim.zero_grad()
            l.backward()
            optim.step()

In [ ]:
start = time.time()
for i in range(train_times):
    gobang = GobangForAI(15,15)
    while gobang.gameover() == False:
        player = gobang.who_should_place()
        board = None
        if player == "Black":
            board = torch.from_numpy(gobang.board_)
        else:
            board = torch.from_numpy(gobang.board_white_)
        x,y = RL.test(net,board,device)
        gobang.place(x,y)
    if gobang.winner == "None":
        continue   
    winner_board,winner_place,_, _ = dataCreator.create(gobang)
    winner_loader = DL.getDataLoader(winner_board,winner_place)
    train(net,winner_loader,True,optim,device,epochs_num)
print(time.time()-start)

In [ ]:
# winner_board,winner_place,loser_board, loser_place = dataCreator.create(gobang)
# print(winner_board.shape,winner_place.shape)
# winner_loader = DL.getDataLoader(winner_board,winner_place)
# loser_loader = DL.getDataLoader(loser_board,loser_place)

In [ ]:
gobang = GobangForAI(15,15)
while gobang.gameover() == False:
    player = gobang.who_should_place()
    board = None
    if player == "Black":
        board = torch.from_numpy(gobang.board_)
    else:
        board = torch.from_numpy(gobang.board_white_)
    x,y = RL.test(net,board,device)
    gobang.place(x,y)
    print(x,y)
    print(gobang.board_)
    a = input()
print(gobang.winner()+" win ")
print(gobang.board_)
print(x,y)